In [1]:
import torch 
import torchvision.transforms as transforms 
from torchvision.datasets import ImageFolder 
 
# 设置随机种子 
torch.manual_seed(42) 
 
# 检查GPU是否可用 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(f"Using device: {device}")

data_dir = "Chinese_Medicine"  # Chinese_Medicine文件夹路径 
image_size = (224, 224)  # 图像大小 
batch_size = 32  # 批大小 
 
# 定义数据预处理的转换操作 
transform = transforms.Compose([ 
    transforms.Resize(image_size), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 
0.224, 0.225]),  # 标准化操作 
]) 
 
# 加载图像数据集 
dataset = ImageFolder(data_dir, transform=transform) 
 
# 获取数据集的类别数量 
num_classes = len(dataset.classes)

# 打印数据集的大小和类别数量 
print("数据集大小:", len(dataset)) 
print("类别数量:", num_classes) 
print("类别:", dataset.classes) 

Using device: cpu
数据集大小: 1048
类别数量: 5
类别: ['baihe', 'baizhi', 'gouqi', 'tianma', 'yiren']


In [2]:
#计算测试集的样本数量 
num_test_samples = int(len(dataset) * 0.3) 
 
#计算训练集的样本数量 
num_train_samples = len(dataset) - num_test_samples 
 
#使用随机分割函数将数据集拆分为训练集和测试集 
train_dataset, test_dataset = torch.utils.data.random_split( 
    dataset, [num_train_samples, num_test_samples]) 
 
#创建数据加载器 
train_loader = torch.utils.data.DataLoader( 
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True) 
 
test_loader = torch.utils.data.DataLoader( 
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False)

In [3]:
import torch.nn as nn 
 
# 定义卷积神经网络模型 
class ConvNet_1(nn.Module): 
    def __init__(self, num_classes): 
        super(ConvNet_1, self).__init__() 
        self.model = nn.Sequential( 
        nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1), 
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size=2, stride=2), 
        nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), 
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size=2, stride=2), 
        nn.Flatten(), 
        nn.Linear(32 * 56 * 56, 256), 
        nn.ReLU(), 
        nn.Linear(256, num_classes) 
    ) 
    def forward(self, x): 
        return self.model(x) 
# 实例化卷积神经网络模型 
model = ConvNet_1(num_classes) 
# 模型移动到指定的设备GPU/CPU 
model.to(device)

ConvNet_1(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=100352, out_features=256, bias=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=5, bias=True)
  )
)

In [4]:
import torch.optim as optim 
#代码8-4 训练模型 
# 定义损失函数和优化器 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 
# 训练模型 
num_epochs = 20 
train_losses = [] 
train_accuracies = [] 
test_accuracies = [] 
for epoch in range(num_epochs): 
    running_loss = 0.0 
    correct_predictions = 0 
    total_predictions = 0 
    for images, labels in train_loader: 
# 将数据移动到设备上 
        images = images.to(device) 
        labels = labels.to(device) 
# 前向传播 
 
 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
 
        # 反向传播和优化 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
         
        # 统计训练过程中的损失值和准确率 
        running_loss += loss.item() 
        _, predicted = torch.max(outputs.data, 1) 
        total_predictions += labels.size(0) 
        correct_predictions += (predicted == labels).sum().item() 
    
    # 计算训练集的平均损失值和准确率 
    train_loss = running_loss / len(train_loader) 
    train_accuracy = correct_predictions / total_predictions 
    train_losses.append(train_loss) 
    train_accuracies.append(train_accuracy) 
    
    # 在每个epoch结束后，在测试集上评估模型 
    model.eval() 
 
    test_correct_predictions = 0 
    test_total_predictions = 0 
 
    for images, labels in test_loader: 
        # 将数据移动到设备上 
        images = images.to(device) 
        labels = labels.to(device) 
 
        # 前向传播 
        outputs = model(images) 
 
        # 统计测试集上的准确率 
        _, predicted = torch.max(outputs.data, 1) 
        test_total_predictions += labels.size(0) 
        test_correct_predictions += (predicted == labels).sum().item() 
 
    # 计算测试集的准确率 
    test_accuracy = test_correct_predictions / test_total_predictions 
    test_accuracies.append(test_accuracy) 
 
    # 打印训练过程中的损失值和准确率 
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

from PIL import Image 
#代码8-7 使用模型进行预测 
#加载预测图片并进行预处理 
test_image_path = 'gouqi.jpg' 
test_image = Image.open(test_image_path).convert('RGB') 
test_image = transform(test_image).unsqueeze(0).to(device) 
#将图片输入模型进行预测 
model.eval() 
with torch.no_grad(): 
    output = model(test_image) 
    _, predicted_idx = torch.max(output, 1) 
#获取预测类别名 
predicted_label = dataset.classes[predicted_idx.item()] 
print(f"Predicted Label: {predicted_label}") 

Epoch [1/20], Train Loss: 3.7423, Train Accuracy: 0.2929, Test Accuracy: 0.3981
Epoch [2/20], Train Loss: 1.2450, Train Accuracy: 0.4796, Test Accuracy: 0.5701
Epoch [3/20], Train Loss: 0.7793, Train Accuracy: 0.7030, Test Accuracy: 0.6210
Epoch [4/20], Train Loss: 0.4239, Train Accuracy: 0.8678, Test Accuracy: 0.6656
Epoch [5/20], Train Loss: 0.2527, Train Accuracy: 0.9332, Test Accuracy: 0.6783
Epoch [6/20], Train Loss: 0.0796, Train Accuracy: 0.9864, Test Accuracy: 0.6624
Epoch [7/20], Train Loss: 0.0423, Train Accuracy: 0.9986, Test Accuracy: 0.6688
Epoch [8/20], Train Loss: 0.0155, Train Accuracy: 1.0000, Test Accuracy: 0.6561
Epoch [9/20], Train Loss: 0.0070, Train Accuracy: 1.0000, Test Accuracy: 0.6433
Epoch [10/20], Train Loss: 0.0036, Train Accuracy: 1.0000, Test Accuracy: 0.6561
Epoch [11/20], Train Loss: 0.0025, Train Accuracy: 1.0000, Test Accuracy: 0.6433
Epoch [12/20], Train Loss: 0.0019, Train Accuracy: 1.0000, Test Accuracy: 0.6401
Epoch [13/20], Train Loss: 0.0016, Tr

In [5]:
import torch.nn as nn 
 
# 定义卷积神经网络模型 
class ConvNet_2(nn.Module): 
    def __init__(self, num_classes): 
        super(ConvNet_2, self).__init__() 
        self.model = nn.Sequential( 
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.BatchNorm2d(16),  # 添加批归一化层 
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.BatchNorm2d(32),  # 添加批归一化层 
            nn.Flatten(), 
            nn.Linear(32 * 56 * 56, 256), 
            nn.ReLU(), 
            nn.BatchNorm1d(256),  # 添加批归一化层 
            nn.Linear(256, num_classes) 
        ) 
 
    def forward(self, x): 
        return self.model(x) 
 
# 实例化卷积神经网络模型 
model = ConvNet_2(num_classes) 
 
# 模型移动到指定的设备GPU/CPU 
model.to(device)

ConvNet_2(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=100352, out_features=256, bias=True)
    (10): ReLU()
    (11): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=256, out_features=5, bias=True)
  )
)

In [8]:
import torch.nn as nn 
#代码8-6 构建VGGNet-16 
from torchvision.models import vgg16 
# 加载预训练的VGG16模型 
model = vgg16(pretrained=True) 
# 修改分类器以匹配类的数量 
model.classifier[6] = nn.Linear(4096, num_classes) 
model.classifier.add_module("7", nn.Softmax(dim=1)) 
# 模型移动到指定的设备GPU/CPU 
model.to(device) 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
from PIL import Image 
#代码8-7 使用模型进行预测 
#加载预测图片并进行预处理 
test_image_path = 'gouqi.jpg' 
test_image = Image.open(test_image_path).convert('RGB') 
test_image = transform(test_image).unsqueeze(0).to(device) 
#将图片输入模型进行预测 
model.eval() 
with torch.no_grad(): 
    output = model(test_image) 
    _, predicted_idx = torch.max(output, 1) 
#获取预测类别名 
predicted_label = dataset.classes[predicted_idx.item()] 
print(f"Predicted Label: {predicted_label}") 

Predicted Label: baihe
